# PUTATIVE WORKFLOW


## WORKFLOW EDITOR PLUGIN
- FINE-TUNE SEGMENTATIONS
  - export workflow.jsons
    - masks:
      - nuclei
      - cellmask
      - cytoplasm
    - organelles:
      - lyso
      - mito
      - golgi
      - perox
      - ER
      - LD


## BATCHPROCESS WORKFLOW
- BATCH PROCESS
  - load workflow.jsons for: 
  1. masks
    - export: masks .tiff as stack (nuclei, cellmask, cytoplasm)
  2. organelles
    - export individual .tiffs



## NOTEBOOK ~~OR ***FUTURE*** PLUGIN~~
- COLLECT ORGANELLE STATS
  - extract masks.tiffs as individual
    - nuclei, cellmask, cytoplasm
  - collect regionprops for all organelles
    - export .csvs


## NOTEBOOK ~~OR __FUTURE__ PLUGIN~~
- SUMMARIZE STUDY DATA
  - munge .csv to create summary stats across all cells/images




_____________

## TO DO
- add "segmentation name" field instead of copying from workflow.json name


- choose alternate conf_XXX.json location. 
  - strategy:  add to "prebuilt" list from path


  
  ## FILE NAME CONVENTIONS

  raw file name is kept.

  PREFIX = "segmentation name" or regionprop name.  e.g. 
  SUFFIX = "description" i.e. 

In [1]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union, Dict, List
import itertools 

import time

import numpy as np
import pandas as pd

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files,
                                        read_tiff_image)



from infer_subc.constants import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats_helpers import *
from infer_subc.utils.stats import _assert_uint16_labels
from infer_subc.core.img import label_uint16


import time
%load_ext autoreload
%autoreload 2



## Get and load image for quantification
Specifically, this will include the raw image and the outputs from segmentation.

In [5]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"

raw_data_path = data_root_path / "raw"
im_type = ".czi"

raw_file_list = list_image_files(raw_data_path,im_type)
raw_img_name = raw_file_list[test_img_n]

# adding an additional list of image paths for the matching segmentation files
seg_data_path = data_root_path / "out"
seg_file_list = list_image_files(seg_data_path, "tiff")

# changing output directory for this notebook to a new folder called "quant"
out_data_path = data_root_path / "quant"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [6]:
raw_im_path = Path(raw_file_list[0])
raw_im_path

WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw/24hrs-Ctrl +oleicAcid 50uM_2_Unmixing.czi')

_______________________

## 1. get each unique cells accouding to filename


### extract ID. e.g.

### process each cell & summarize



## 2. aggregate all cells into a database 

In [ ]:
full_name = raw_im_path.name

cell_ids = [ Path(fn).stem.split("-")[0] for fn in raw_file_list]
cell_ids = list(set(cell_ids))

masks_postfix = "masks"
organelle_postfix = ["lyso","mito","golgi","perox","ER","LD"]


## 3. Explode masks files into individual files

In [ ]:
# TODO: depricate this an integrate importing the masks as one file that is utilized one peice at a ttime.


def _explode_mask(mask_path: Union[Path,str], postfix: str= "masks", im_type: str = ".tiff") -> bool:
    """ 
    TODO: add loggin instead of printing
        append tiffcomments with provenance
    """
    if isinstance(mask_path, str): mask_path = Path(mask_path)
    # load image 
    full_stem = mask_path.stem
    if full_stem.endswith(postfix):
        stem = full_stem.rstrip(postfix)
        image = read_tiff_image(mask_path)
        assert image.shape[0]==3
        
        # make into np.uint16 labels
        nuclei = label_uint16(image[0])
        # export as np.uint8 (255)
        cellmask = image[1]>0            
        cytoplasm = image[2]>0

        # write wasks
        root_stem = mask_path.parent / stem
        # ret1 = imwrite(f"{root}nuclei{stem}", nuclei)
        ret1 = export_tiff(nuclei, f"{stem}nuc", mask_path.parent, None)
        # ret2 = imwrite(f"{root}cellmask{stem}", cellmask)
        ret2 = export_tiff(cellmask, f"{stem}cell", mask_path.parent, None)
        # ret3 = imwrite(f"{root}cytosol{stem}", cytosol)
        ret3 = export_tiff(cytoplasm, f"{stem}cyto", mask_path.parent, None)

        print(f"wrote {stem}-{{nuc,cell,cyto}}")
        return True
    else:
        return False



def _explode_masks(root_path: Union[Path,str], postfix: str= "masks", im_type: str = ".tiff"):
    """  
    TODO: add loggin instead of printing
        append tiffcomments with provenance
    """
    if isinstance(root_path, str): root_path = Path(root_path)
    img_file_list = list_image_files(root_path,im_type, postfix)
    wrote_cnt = 0
    for img_f in img_file_list:
        if _explode_mask(img_f, postfix=postfix, im_type=im_type): wrote_cnt += 1
        else: print(f"failed to explode {img_f}")
    else:
        print(f"how thefark!!! {img_f}")

    print(f"exploded {wrote_cnt*100./len(img_file_list)} pct of {len(img_file_list)} files")
    return wrote_cnt



In [ ]:
from infer_subc.utils.batch import explode_masks

cnt = explode_masks(seg_data_path, postfix='masks')
cnt

## 4. Acquire the segmentation files

In [7]:
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
# linearly unmixed ".czi" files are here
raw_data_path = data_root_path / "raw"
# save output ".tiff" files here
int_data_path = data_root_path / "out"
# save stats here
out_data_path = data_root_path / "quant"


In [ ]:
raw_path = raw_data_path
int_path = int_data_path
out_path = out_data_path


if isinstance(raw_path, str): raw_path = Path(raw_path)
if isinstance(int_path, str): int_path = Path(int_path)
if isinstance(out_path, str): out_path = Path(out_path)

img_file_list = list_image_files(raw_path,".czi")

if not Path.exists(out_path):
    Path.mkdir(out_path)
    print(f"making {out_path}")



In [ ]:
img_file_list

In [8]:
from typing import Optional, Union, Dict, List
from infer_subc.utils.batch import explode_mask

test_suffix = "-20230426_test_"

def _find_segmentation_tiff_files(prototype:Union[Path,str],
                                  name_list: List[str], 
                                  int_path: Union[Path,str],
                                  suffix: Union[str, None] = None ) -> Dict:
    """
    find the nescessary image files based on protype, the organelles involved, and paths

    suffix:
        the str that comes between the .stem of the file name and the organelle/cell area type sufffix.
        Ex) Prototype = "C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw/a48hrs-Ctrl_9_Unmixing.czi"
            Name of organelle file = a48hrs-Ctrl_9_Unmixing-20230426_test_cell.tiff
            result of .stem = "a48hrs-Ctrl_9_Unmixing"
            organelle/cell area type = "cell"
            suffix = "-20230426_test_"

    """

    # raw
    prototype = Path(prototype)
    if not prototype.exists():
        print(f"bad prototype. please choose an existing `raw` file as prototype")
        return dict()
    # make sure protoype ends with czi

    out_files = {"raw":prototype}

    int_path = Path(int_path) 
    # raw
    if not int_path.is_dir():
        print(f"bad path argument. please choose an existing path containing organelle segmentations")
        return out_files
    
    # # cyto, cellmask
    # cyto_nm = int_path / f"{prototype.stem}{suffix}cyto.tiff"
    # if cyto_nm.exists():
    #     out_files["cyto"] = cyto_nm
    # else:
    #     print(f"cytosol mask not found.  We'll try to extract from masks ")
    #     if explode_mask(int_path / f"{prototype.stem}{suffix}masks.tiff"): 
    #         out_files["cyto"] = cyto_nm
    #     else: 
    #         print(f"failed to explode {prototype.stem}{suffix}masks.tiff")
    #         return out_files
    
    # cellmask_nm = int_path / f"{prototype.stem}{suffix}cell.tiff"
    # if  cellmask_nm.exists():
    #     out_files["cell"] = cellmask_nm
    # else:
    #     print(f"cell file not found in {int_path} returning")
    #     out_files["cell"] = None

    # nuc_nm = int_path / f"{prototype.stem}{suffix}nuc.tiff"
    # if  cellmask_nm.exists():
    #     out_files["nuc"] = cellmask_nm
    # else:
    #     print(f"nuc file not found in {int_path} returning")
    #     out_files["nuc"] = None

    # organelles
    for org_n in name_list:
        org_name = Path(int_path) / f"{prototype.stem}{suffix}{org_n}.tiff"
        if org_name.exists(): 
            out_files[org_n] = org_name
        else: 
            print(f"{org_n} .tiff file not found in {int_path} returning")
            out_files[org_n] = None
    
    return out_files

    


In [9]:
from infer_subc.utils.batch import find_segmentation_tiff_files
prototype = raw_file_list[0]
organelles = ["lyso","mito","golgi","perox","ER","LD","cell", "cyto", "nuc"]

filez = _find_segmentation_tiff_files(prototype, organelles, int_data_path, test_suffix)



In [10]:
filez

{'raw': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw/24hrs-Ctrl +oleicAcid 50uM_2_Unmixing.czi'),
 'lyso': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out/24hrs-Ctrl +oleicAcid 50uM_2_Unmixing-20230426_test_lyso.tiff'),
 'mito': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out/24hrs-Ctrl +oleicAcid 50uM_2_Unmixing-20230426_test_mito.tiff'),
 'golgi': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out/24hrs-Ctrl +oleicAcid 50uM_2_Unmixing-20230426_test_golgi.tiff'),
 'perox': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out/24hrs-Ctrl +oleicAcid 50uM_2_Unmixing-20230426_test_perox.tiff'),
 'ER': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out/24hrs-Ctrl +oleicAcid 50uM_2_Unmixing-20230426_test_ER.tiff'),
 'LD': WindowsPath('C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/out/24hrs-Ctrl +oleicAcid 50uM_2_Unmixing-20230426_test_LD.tiff'),
 'cell': WindowsP

In [12]:
region_names = ["cell", "cyto", "nuc"]

test_regions = []
for name in region_names:
    test_mask = read_tiff_image(filez[name])
    print(name)
    test_regions.append(test_mask)

cell
cyto
nuc


In [26]:
type(region_names)

list

In [27]:
region_names.index('nuc')

2

In [28]:
test_regions[region_names.index('nuc')]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

## 5. Batch process the data from multiple images at one time

In [2]:
from infer_subc.utils.stats_helpers import make_all_metrics_tables, dump_all_metrics_tables
from infer_subc.constants import *
from infer_subc.organelles import *

# names of organelles we have
organelle_names = ["lyso", "mito","golgi","perox","ER","LD"]
region_names = ["cell", "cyto", "nuc"]

# get the intensities
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]



In [4]:
new_list = organelle_names + region_names
print(new_list)

['lyso', 'mito', 'golgi', 'perox', 'ER', 'LD', 'cell', 'cyto', 'nuc', 'cell', 'cyto', 'nuc']


In [9]:
# for a list of "prefixes"  collect stats + cross stats masked by cytosol (including nuclei masked by cellmask)

# NOTE: the convex hull regionprops error is a know issue that occurs when the objects being measured have too few voxels. 
# Here's the github reference:https://github.com/scikit-image/scikit-image/issues/5363

# TODO: it may be beneficial in the future to make the analysis possible without the analysis of the regions, but for now, it can stay


def _batch_process_quantification(out_file_prefix: str,
                                  seg_path: Union[Path,str],
                                  out_path: Union[Path, str], 
                                  raw_path: Union[Path,str], 
                                  raw_file_type: str,
                                  organelle_names: List[str], 
                                  organelle_chs: List[int], 
                                  region_names: List[str],
                                  seg_suffix: Union[str, None] = None,
                                  scale: bool = True,
                                  n_XY_bins: Union[int,None] = 5,
                                  n_zernike: Union[int,None] = 9,
                                  include_contact_dist: bool = True
                                    ) -> int :
    """  
    TODO: add loggin instead of printing
        append tiffcomments with provenance

    organelle_names - should match the file suffix; I think the order will specify the order in the csv files
        
    """
    start = time.time()
    if isinstance(raw_path, str): raw_path = Path(raw_path)
    if isinstance(seg_path, str): seg_path = Path(seg_path)
    if isinstance(out_path, str): out_path = Path(out_path)
    
    img_file_list = list_image_files(raw_path,raw_file_type)

    if not Path.exists(out_path):
        Path.mkdir(out_path)
        print(f"making {out_path}")
    
    count = 0

    org_tabs = []
    contact_tabs = []
    dist_tabs = []
    region_tabs = []
    
    segs_to_collect = organelle_names + region_names

    if scale == True:
        for img_f in img_file_list:
            count = count + 1
            filez = find_segmentation_tiff_files(img_f, segs_to_collect, seg_path, seg_suffix)
            img_data, meta_dict = read_czi_image(filez["raw"])
            scale = meta_dict['scale'] # this will only work if the scale can be pulled from the metadata this way

            # create intensities from raw as list
            intensities = [img_data[ch] for ch in organelle_chs]

            # load regions
            regions = []
            for name in region_names:
                mask = read_tiff_image(filez[name])
                regions.append(mask)

            nuc_mask = regions[region_names.index('nuc')]
            cell_mask = regions[region_names.index('cell')]

            # load organelles as list
            organelles = [read_tiff_image(filez[org]) for org in organelle_names]

            if scale == True:
                org_metrics, contact_metrics, dist_metrics, region_metrics = make_all_metrics_tables(organelle_names=organelle_names,
                                                                                    organelles=organelles,
                                                                                    intensities=intensities,
                                                                                    regions=regions,
                                                                                    region_names=region_names,
                                                                                    dist_centering_obj=nuc_mask, 
                                                                                    mask=cell_mask,
                                                                                    source_file=img_f,
                                                                                    scale=scale,
                                                                                    n_XY_bins=n_XY_bins,
                                                                                    n_zernike=n_zernike,
                                                                                    include_contact_dist=include_contact_dist)
            else:
                org_metrics, contact_metrics, dist_metrics, region_metrics = make_all_metrics_tables(organelle_names=organelle_names,
                                                                                    organelles=organelles,
                                                                                    intensities=intensities,
                                                                                    regions=regions,
                                                                                    region_names=region_names,
                                                                                    dist_centering_obj=nuc_mask, 
                                                                                    mask=cell_mask,
                                                                                    source_file=img_f,
                                                                                    n_XY_bins=n_XY_bins,
                                                                                    n_zernike=n_zernike,
                                                                                    include_contact_dist=include_contact_dist)
            org_tabs.append(org_metrics)
            contact_tabs.append(contact_metrics)
            dist_tabs.append(dist_metrics)
            region_tabs.append(region_metrics)
            end2 = time.time()
            print(f"Completed processing for {count} images in {(end2-start)/60} mins.")

    final_org = pd.concat(org_tabs, ignore_index=True)
    final_contact = pd.concat(contact_tabs, ignore_index=True)
    final_dist = pd.concat(dist_tabs, ignore_index=True)
    final_region = pd.concat(region_tabs, ignore_index=True)

    org_csv_path = out_path / f"{out_file_prefix}organelles.csv"
    final_org.to_csv(org_csv_path)

    contact_csv_path = out_path / f"{out_file_prefix}contacts.csv"
    final_contact.to_csv(contact_csv_path)

    dist_csv_path = out_path / f"{out_file_prefix}distributions.csv"
    final_dist.to_csv(dist_csv_path)

    region_csv_path = out_path / f"{out_file_prefix}regions.csv"
    final_region.to_csv(region_csv_path)

    end = time.time()
    print(f"Quantification for {count} files is COMPLETE! Files saved to '{out_data_path}'.")
    print(f"It took {(end - start)/60} minutes to quantify these files.")
    return count

In [10]:
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"
# linearly unmixed ".czi" files are here
raw_data_path = data_root_path / "raw"
# save output ".tiff" files here
seg_data_path = data_root_path / "out"
# save stats here
out_data_path = data_root_path / "quant"


# names of organelles we have
organelle_names = ["lyso", "mito","golgi","perox","ER","LD"]
region_names = ["cell", "cyto", "nuc"]

# get the intensities
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]

In [11]:
n_files = _batch_process_quantification(out_file_prefix="20230922_TEST_withregions_",
                             seg_path=seg_data_path,
                             out_path=out_data_path, 
                             raw_path=raw_data_path, 
                             raw_file_type='.czi',
                             organelle_names=organelle_names, 
                             organelle_chs=organelle_channels, 
                             region_names=region_names,
                             seg_suffix="-20230426_test_",
                             scale=True,
                             n_XY_bins=5,
                             n_zernike=9,
                             include_contact_dist=True)

c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1979497671  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  7  Error-roundoff 9.7e-15  _one-merge 6.8e-14
  _near-inside 3.4e-13  Visible-distance 1.9e-14  U-max-coplanar 1.9e-14
  Width-outside 3.9e-14  _wide-facet 1.2e-13  _maxoutside 7.8e-14

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While ex

It took 1.5885069012641906 minutes to quantify one image.
Completed processing for 1 images in 1.5940137465794881 mins.


c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1981111143  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 11  Error-roundoff 1.5e-14  _one-merge 1e-13  _near-inside 5.1e-13
  Visible-distance 2.9e-14  U-max-coplanar 2.9e-14  Width-outside 5.8e-14
  _wide-facet 1.7e-13  _maxoutside 1.2e-13

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While exec

It took 1.6854422688484192 minutes to quantify one image.
Completed processing for 2 images in 3.285200027624766 mins.


c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1982825457  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  8  Error-roundoff 1.1e-14  _one-merge 7.8e-14
  _near-inside 3.9e-13  Visible-distance 2.2e-14  U-max-coplanar 2.2e-14
  Width-outside 4.4e-14  _wide-facet 1.3e-13  _maxoutside 8.9e-14

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While ex

...

It took 1.14005735317866 minutes to quantify one image.
Completed processing for 12 images in 23.38191358645757 mins.


c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 2003128313  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  4  Error-roundoff 5.5e-15  _one-merge 3.9e-14
  _near-inside 1.9e-13  Visible-distance 1.1e-14  U-max-coplanar 1.1e-14
  Width-outside 2.2e-14  _wide-facet 6.7e-14  _maxoutside 4.4e-14

  return convex_hull_image(self.image)
c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\skimage\measure\_regionprops.py:431: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While ex

It took 2.084163943926493 minutes to quantify one image.
Completed processing for 13 images in 25.471696599324545 mins.


c:\Users\Shannon\anaconda3\envs\infer-subc\lib\site-packages\pandas\core\internals\blocks.py:2536: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


Quantification for 13 files is COMPLETE! Files saved to 'C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\quant'.
It took 25.475935248533883 minutes to quantify these files.
